In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from sklearn.model_selection import ParameterGrid

from banditrec.datasets import InteractionDataset, import_movielens, import_lastfm, import_delicious, import_delicious2, import_lastfm2
from banditrec.simulate import SimulatorEC, SimulatorRP, SimulatorSW
from banditrec.agents import RandomAgent, HindsightAgent, UCBAgent, CLUBAgent, DynUCBAgent, NUCBAgent, LinUCBAgent, NLinUCBAgent, SCLUBAgent, UserNeighborAgent
from banditrec.tuning import tune

This notebook runs the hyper-parameter tuning code and final experiments comparing NUCB with state of the art algorithms.

It assumes a the datasets are located in the directory `data/` and that the directory`experiment1/raw/` exists where the simulation results will be stored.

## Load dataset and helper functions

In [ ]:
BASE_PATH = "./data"

R_lastfm, _ = import_lastfm(f"{BASE_PATH}/hetrec2011-lastfm-2k/user_artists.dat", K=1000)
R_lastfmx1, features_lastfmx1 = import_lastfm2(f"{BASE_PATH}/hetrec2011-lastfm-2k", d=25)
R_lastfmx2, features_lastfmx2 = import_lastfm2(f"{BASE_PATH}/hetrec2011-lastfm-2k", d=25, K=1000)
R_delicious, _ = import_delicious(f"{BASE_PATH}/hetrec2011-delicious-2k", K=1000)
R_deliciousx1, features_deliciousx1 = import_delicious2(f"{BASE_PATH}/hetrec2011-delicious-2k", d=25)
R_deliciousx2, features_deliciousx2 = import_delicious2(f"{BASE_PATH}/hetrec2011-delicious-2k", d=25, K=1000)
R_movielens, _ = import_movielens(f"{BASE_PATH}/ml-100k/u.data", '\t')

dataset_lastfm = InteractionDataset(R_lastfm, "LastFM", 200000)
dataset_lastfmx1 = InteractionDataset(R_lastfmx1, "LastFMx1", 50000).add_item_features(features_lastfmx1)
dataset_lastfmx2 = InteractionDataset(R_lastfmx2, "LastFMx2", 100000).add_item_features(features_lastfmx2)
dataset_delicious = InteractionDataset(R_delicious, "Delicious", 200000)
dataset_deliciousx1 = InteractionDataset(R_deliciousx1, "Deliciousx1", 50000).add_item_features(features_deliciousx1)
dataset_deliciousx2 = InteractionDataset(R_deliciousx2, "Deliciousx2", 100000).add_item_features(features_deliciousx2)
dataset_movielens = InteractionDataset(R_movielens>=4, "Movielens", 200000)

run_count = 5
tuning_run_count = 3
tuning_fraction = 0.2
pool_size = 25

def tune_generic(simulator, agent, grid, filename):
    simulator.set_run_count(tuning_run_count)
    simulator.set_tqdm({"leave": False})
    dataset = simulator.dataset
    horizon = math.floor(tuning_fraction * dataset.time_horizon)
    
    tune2(simulator, agent, grid, horizon, filename)

def run_generic(simulator, agent, params, filename):
    simulator.set_run_count(run_count)
    dataset = simulator.dataset
    a = agent(**params)
    
    if dataset.item_features is not None and agent.__name__ not in ["RandomAgent", "HindsightAgent"]:
        a.set_features(dataset.item_features)
    
    result = simulator.run(a)
    np.save(filename, result.rewards)
    
def tuneEC(dataset, agent, grid):
    sim = SimulatorEC(dataset)
    tune_generic(sim, agent, grid, "experiment1/tuningEC.txt")

def tuneRP(dataset, agent, grid):
    sim = SimulatorRP(dataset, pool_size)
    tune_generic(sim, agent, grid, "experiment1/tuningRP.txt")

def tuneSW(dataset, agent, grid):
    sim = SimulatorSW(dataset, pool_size)
    tune_generic(sim, agent, grid, "experiment1/tuningSW.txt")

def runEC(dataset, agent, params, name):
    sim = SimulatorEC(dataset)
    run_generic(sim, agent, grid, f"experiment1/raw/{dataset.name}EC_{name}.npy")

def runRP(dataset, agent, params, name):
    sim = SimulatorRP(dataset, pool_size)
    run_generic(sim, agent, params, f"experiment1/raw/{dataset.name}RP_{name}.npy")

def runSW(dataset, agent, params, name):
    sim = SimulatorSW(dataset, pool_size)
    run_generic(sim, agent, params, f"experiment1/raw/{dataset.name}SW_{name}.npy")

def create_tunable():
    alphas = [0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2]

    ucb = (UCBAgent, ParameterGrid({
        "alpha": alphas
    }))
    NUCB = (NUCBAgent, ParameterGrid({
        "alpha": alphas,
        "uw": [100],
        "iw": [100]
    }))
    dynucb = (DynUCBAgent, ParameterGrid({
        "alpha": alphas,
        "cluster_count": [1, 5, 10, 25] 
    }))
    linucb1 = (LinUCBAgent, ParameterGrid({
        "alpha": alphas,
        "personalized": [True]
    }))
    linucb2 = (LinUCBAgent, ParameterGrid({
        "alpha": alphas,
        "personalized": [False]
    }))
    NLinUCB = (NLinUCBAgent, ParameterGrid({
        "alpha": alphas,
        "uw": [100],
    }))
    club = (CLUBAgent, ParameterGrid({
        "alpha": alphas,
        "beta": [0.25, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
    }))
    sclub = (SCLUBAgent, ParameterGrid({
        "beta": alphas,
        "alpha1": [1, 2, 3, 4, 5],
        "alpha2": [1, 2, 3, 4, 5],
    }))
    un = (UserNeighborAgent, ParameterGrid({
        "prior": [(1, 1), (1, 10), (1, 100)]
    }))

    return {
        "ucb": ucb,
        "NUCB": NUCB,
        "cab": cab,
        "dynucb": dynucb,
        "linucb1": linucb1,
        "linucb2": linucb2,
        "NLinUCB": NLinUCB,
        "club": club,
        "sclub": sclub,
        "un": un
    }

## Parameter tuning

In [ ]:
agents = create_tunable()

# lastfm
d = dataset_lastfm
tuneRP(d, *agents["ucb"])
tuneRP(d, *agents["club"])
tuneRP(d, *agents["dynucb"])
tuneRP(d, *agents["sclub"])
tuneRP(d, *agents["un"])
tuneRP(d, *agents["NUCB"])
tuneSW(d, *agents["ucb"])
tuneSW(d, *agents["club"])
tuneSW(d, *agents["dynucb"])
tuneSW(d, *agents["sclub"])
tuneSW(d, *agents["un"])
tuneSW(d, *agents["NUCB"])

# delicious
d = dataset_delicious
tuneRP(d, *agents["ucb"])
tuneRP(d, *agents["club"])
tuneRP(d, *agents["dynucb"])
tuneRP(d, *agents["sclub"])
tuneRP(d, *agents["un"])
tuneRP(d, *agents["NUCB"])
tuneSW(d, *agents["ucb"])
tuneSW(d, *agents["club"])
tuneSW(d, *agents["dynucb"])
tuneSW(d, *agents["sclub"])
tuneSW(d, *agents["un"])
tuneSW(d, *agents["NUCB"])

# lastfmx
d = dataset_lastfmx1
tuneRP(d, *agents["linucb1"])
tuneRP(d, *agents["linucb2"])
tuneRP(d, *agents["club"])
tuneRP(d, *agents["dynucb"])
tuneRP(d, *agents["sclub"])
tuneRP(d, *agents["NLinUCB"])
d = dataset_lastfmx2
tuneSW(d, *agents["linucb1"])
tuneSW(d, *agents["linucb2"])
tuneSW(d, *agents["club"])
tuneSW(d, *agents["dynucb"])
tuneSW(d, *agents["sclub"])
tuneSW(d, *agents["NLinUCB"])

# deliciousx
d = dataset_deliciousx1
tuneRP(d, *agents["linucb1"])
tuneRP(d, *agents["linucb2"])
tuneRP(d, *agents["club"])
tuneRP(d, *agents["dynucb"])
tuneRP(d, *agents["sclub"])
tuneRP(d, *agents["NLinUCB"])
d = dataset_deliciousx2
tuneSW(d, *agents["linucb1"])
tuneSW(d, *agents["linucb2"])
tuneSW(d, *agents["club"])
tuneSW(d, *agents["dynucb"])
tuneSW(d, *agents["sclub"])
tuneSW(d, *agents["NLinUCB"])

# movielens
d = dataset_movielens
tuneRP(d, *agents["ucb"])
tuneRP(d, *agents["club"])
tuneRP(d, *agents["dynucb"])
tuneRP(d, *agents["sclub"])
tuneRP(d, *agents["NUCB"])
tuneRP(d, *agents["un"])
tuneSW(d, *agents["ucb"])
tuneSW(d, *agents["club"])
tuneSW(d, *agents["dynucb"])
tuneSW(d, *agents["sclub"])
tuneSW(d, *agents["NUCB"])
tuneSW(d, *agents["un"])

## Running full simulation
The parameter values below were obtained from the code in the previous cell

In [ ]:
# lastfm
d = dataset_lastfm
runRP(d, RandomAgent, {}, "Random")
runRP(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runRP(d, UCBAgent, {"alpha": 0.1}, "UCB")
runRP(d, CLUBAgent, {"alpha": 0.06, "beta": 4.5}, "CLUB")
runRP(d, DynUCBAgent, {"alpha": 0.1, "cluster_count": 1}, "DynUCB")
runRP(d, SCLUBAgent, {"beta": 0.06, "alpha1": 4, "alpha2": 2}, "SCLUB")
runRP(d, UserNeighborAgent, {"prior": [1, 100]}, "UN")
runRP(d, NUCBAgent, {"alpha": 0.04, "uw": 100, "iw": 100}, "NUCB")

runSW(d, RandomAgent, {}, "Random")
runSW(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runSW(d, UCBAgent, {"alpha": 0.06}, "UCB")
runSW(d, CLUBAgent, {"alpha": 0.06,  "beta": 1.5}, "CLUB")
runSW(d, DynUCBAgent, {"alpha": 0.08, "cluster_count": 1}, "DynUCB")
runSW(d, SCLUBAgent, {"beta": 0.08, "alpha1": 4, "alpha2": 4}, "SCLUB")
runSW(d, UserNeighborAgent, {"prior": [1, 1]}, "UN")
runSW(d, NUCBAgent, {"alpha": 0.08,  "uw": 100, "iw": 100}, "NUCB")

# delicious
d  = dataset_delicious
runRP(d, RandomAgent, {}, "Random")
runRP(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runRP(d, UCBAgent, {"alpha": 0.12}, "UCB")
runRP(d, CLUBAgent, {"alpha": 0.12, "beta": 4.5}, "CLUB")
runRP(d, DynUCBAgent, {"alpha": 0.14, "cluster_count": 1}, "DynUCB")
runRP(d, SCLUBAgent, {"beta": 0.1, "alpha1": 5, "alpha2": 1}, "SCLUB")
runRP(d, NUCBAgent, {"alpha": 0.06, "uw": 100, "iw": 100}, "NUCB")
runRP(d, UserNeighborAgent, {"prior": [1, 100]}, "UN2")

runSW(d, RandomAgent, {}, "Random")
runSW(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runSW(d, UCBAgent, {"alpha": 0.04}, "UCB")
runSW(d, DynUCBAgent, {"alpha": 0.04, "cluster_count": 1}, "DynUCB")
runSW(d, CLUBAgent, {"alpha": 0.04,  "beta": 1.5}, "CLUB")
runSW(d, SCLUBAgent, {"beta": 0.04, "alpha1": 5, "alpha2": 2}, "SCLUB")
runSW(d, NUCBAgent, {"alpha": 0.04,  "uw": 100, "iw": 100}, "NUCB")
runSW(d, UserNeighborAgent, {"prior": [1, 1]}, "UN2")

# lastfmx
d = dataset_lastfmx1
runRP(d, RandomAgent, {}, "Random")
runRP(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runRP(d, LinUCBAgent, {"personalized": True, "alpha": 0.06}, "MUL-LinUCB")
runRP(d, LinUCBAgent, {"personalized": False, "alpha": 0.08}, "ONE-LinUCB")
runRP(d, CLUBAgent, {"alpha": 0.12, "beta": 0.75}, "CLUB")
runRP(d, DynUCBAgent, {"alpha": 0.04, "cluster_count": 1}, "DynUCB")
runRP(d, SCLUBAgent, {"beta": 0.1, "alpha1": 2, "alpha2": 2}, "SCLUB")
runRP(d, NLinUCBAgent, {"alpha": 0.08, "uw": 100}, "NLinUCB")
d = dataset_lastfmx2
runSW(d, RandomAgent, {}, "Random")
runSW(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runSW(d, LinUCBAgent, {"personalized": True, "alpha": 0.18}, "MUL-LinUCB")
runSW(d, LinUCBAgent, {"personalized": False, "alpha": 0.1}, "ONE-LinUCB")
runSW(d, CLUBAgent, {"alpha": 0.14, "beta": 1.5}, "CLUB")
runSW(d, DynUCBAgent, {"alpha": 0.1, "cluster_count": 1}, "DynUCB")
runSW(d, SCLUBAgent, {"beta": 0.14, "alpha1": 4, "alpha2": 1}, "SCLUB")
runSW(d, NLinUCBAgent, {"alpha": 0.16,  "uw": 100}, "NLinUCB")

# deliciousx
d = dataset_deliciousx1
runRP(d, RandomAgent, {}, "Random")
runRP(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runRP(d, LinUCBAgent, {"personalized": True, "alpha": 0.02}, "MUL-LinUCB")
runRP(d, LinUCBAgent, {"personalized": False, "alpha": 0.04}, "ONE-LinUCB")
runRP(d, CLUBAgent, {"alpha": 0.1, "beta": 0.5}, "CLUB")
runRP(d, DynUCBAgent, {"alpha": 0.02, "cluster_count": 20}, "DynUCB")
runRP(d, SCLUBAgent, {"beta": 0.02, "alpha1": 1, "alpha2": 4}, "SCLUB")
runRP(d, NLinUCBAgent, {"alpha": 0.08, "uw": 100}, "NLinUCB")
d = dataset_lastfmx2
runSW(d, RandomAgent, {}, "Random")
runSW(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runSW(d, LinUCBAgent, {"personalized": True, "alpha": 0.12}, "MUL-LinUCB")
runSW(d, LinUCBAgent, {"personalized": False, "alpha": 0.06}, "ONE-LinUCB")
runSW(d, CLUBAgent, {"alpha": 0.08, "beta": 5}, "CLUB")
runSW(d, DynUCBAgent, {"alpha": 0.04, "cluster_count": 25}, "DynUCB")
runSW(d, SCLUBAgent, {"beta": 0.1, "alpha1": 4, "alpha2": 4}, "SCLUB")
runSW(d, NLinUCBAgent, {"alpha": 0.18,  "uw": 100}, "NLinUCB")

# movielens
runRP(d, RandomAgent, {}, "Random")
runRP(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runRP(d, UCBAgent, {"alpha": 0.08}, "UCB")
runRP(d, CLUBAgent, {"alpha": 0.1, "beta": 4.5}, "CLUB")
runRP(d, DynUCBAgent, {"alpha": 0.06, "cluster_count": 5}, "DynUCB")
runRP(d, SCLUBAgent, {"beta": 0.02, "alpha1": 5, "alpha2": 2}, "SCLUB")
runRP(d, NUCBAgent, {"alpha": 0.02, "uw": 100, "iw": 100}, "NUCB")
runRP(d, UserNeighborAgent, {"prior": [1, 1]}, "UN")

runSW(d, RandomAgent, {}, "Random")
runSW(d, HindsightAgent, {"ranked_items": d.ranked_items}, "Hindsight")
runSW(d, UCBAgent, {"alpha": 0.1}, "UCB")
runSW(d, CLUBAgent, {"alpha": 0.06,  "beta": 4.5}, "CLUB")
runSW(d, DynUCBAgent, {"alpha": 0.08, "cluster_count": 1}, "DynUCB")
runSW(d, SCLUBAgent, {"beta": 0.1, "alpha1": 5, "alpha2": 2}, "SCLUB")
runSW(d, NUCBAgent, {"alpha": 0.06,  "uw": 100, "iw": 100}, "NUCB")
runSW(d, UserNeighborAgent, {"prior": [1, 10]}, "UN")

## Visualize results

In [ ]:
vis = ExperimentVisualizer2("experiment1/raw", "experiment1/plots")
md = 12

# semi-contextual
vis.new_plot([["lastfmRP", "deliciousRP"], ["lastfmSW", "deliciousSW"]], 200_000)
vis.add_result("Random", {"color": "tab:blue"})
vis.add_result("CLUB",   {"color": "tab:red",    "marker": "^", "markevery": (1*md, 6*md)})
vis.add_result("UCB",    {"color": "tab:green",  "marker": "v", "markevery": (0*md, 6*md)})
vis.add_result("SCLUB",  {"color": "tab:gray",   "marker": ">", "markevery": (3*md, 6*md)})
vis.add_result("DynUCB", {"color": "tab:brown",  "marker": "<", "markevery": (2*md, 6*md)})
vis.add_result("NUCB",   {"color": "tab:purple", "marker": "o", "markevery": (4*md, 6*md)})
vis.add_result("UN",     {"label": "NBandit", "color": "tab:cyan", "marker": "x", "markevery": (5*md, 6*md)})
vis.create_plot("rewards", "semicontextual", [["LastFM RP", "Delicious RP"], ["LastFM SW", "Delicious SW"]])

# contextual
vis.new_plot([["lastfmx1RP", "deliciousx1RP"], ["lastfmx2SW", "deliciousx2SW"]], 50_000, None, True)
vis.add_result("Random",     {"color": "tab:blue"})
vis.add_result("CLUB",       {"color": "tab:red",    "marker": "^", "markevery": (1*md, 5*md)})
vis.add_result("ONE-LinUCB", {"color": "tab:green",  "marker": "v", "markevery": (0*md, 5*md)})
vis.add_result("MUL-LinUCB", {"color": "tab:green",  "marker": "v", "markevery": (0*md, 5*md), "linestyle": "dotted"})
vis.add_result("DynUCB",     {"color": "tab:brown",  "marker": "<", "markevery": (2*md, 5*md)})
vis.add_result("SCLUB",      {"color": "tab:gray",   "marker": ">", "markevery": (3*md, 5*md)})
vis.add_result("NLinUCB",    {"color": "tab:purple", "marker": "o", "markevery": (4*md, 5*md)})
vis.create_plot(["regrets", "rewards"], "contextual", [["LastFM RP", "Delicious RP"], ["LastFM SW", "Delicious SW"]])